# Forced convection of a bank of tubes in cross flow

Pressurized water is often available at elevated temperatures and may be used for space heating
or industrial process applications. In such cases it is customary to use a tube bundle in which
the water is passed through the tubes, while air is passed in cross flow over the tubes. Consider
 a staggered arrangement for which the tube outside diameter is 16.4 mm and the longitudinal and
 transverse pitches are SL = 34.3 mm and ST = 31.3 mm. There are seven rows of tubes in the
 airflow direction and eight tubes per row. Under typical operating conditions the cylinder
 surface temperature is at 70oC, while the air upstream temperature and velocity are 15oC and
 6m/s, respectively. Determine the air-side convection coefficient and the rate of heat transfer
 for the tube bundle.

## Known

In [1]:
import numpy as np

from thermal_system_calculation import FluidState, Fluid

diameter_tube = 16.4e-3 # m
vertical_spacing = 31.3e-3 # m
horizontal_spacing = 34.3e-3 #m
temp_surface = 70 # degC
temp_air_in = 15 # degC
velocity_air = 6 # m/s

## Find

In [2]:
h_air = None # convection coefficient
q = None # Heat transfer for the tube bundle

## Schematic
![schematic](images/TubeBankExample.png)

## Assumptions
1. Steady-state, incompressible flow conditions
2. Radiation neglected
3. Change in properties of air due to the temperature drop over the tube is neglected.

## Properties

In [3]:
fluid_state_air_mean = FluidState(
    fluid=Fluid.AIR,
    temp_k=np.mean([temp_surface, temp_air_in]) + 273.15,
    characteristic_length=diameter_tube
)
print("The properties of air at the mean temperature are:")
print(f"\t- density: {fluid_state_air_mean.density:.3f} kg/m3")
print(f"\t- thermal conductivity: {fluid_state_air_mean.k:.3f} W/mK")
print(f"\t- dynamic viscosity: {fluid_state_air_mean.dynamic_viscosity:.3e} Pa s")
print(f"\t- specific heat capacity: {fluid_state_air_mean.cp} J/kgK")

fluid_state_air_surface = FluidState(
    fluid=Fluid.AIR,
    temp_k=temp_surface + 273.15,
    characteristic_length=diameter_tube
)
print("The properties of air at the surface temperature are:")
print(f"\t- density: {fluid_state_air_surface.density:.3f} kg/m3")
print(f"\t- thermal conductivity: {fluid_state_air_surface.k:.3f} W/mK")
print(f"\t- dynamic viscosity: {fluid_state_air_surface.dynamic_viscosity:.3e} Pa s")
print(f"\t- specific heat capacity: {fluid_state_air_surface.cp} J/kgK")

The properties of air at the mean temperature are:
	- density: 1.118 kg/m3
	- thermal conductivity: 0.028 W/mK
	- dynamic viscosity: 1.928e-05 Pa s
	- specific heat capacity: 1007.0404483923909 J/kgK
The properties of air at the surface temperature are:
	- density: 1.029 kg/m3
	- thermal conductivity: 0.030 W/mK
	- dynamic viscosity: 2.056e-05 Pa s
	- specific heat capacity: 1008.6990251132372 J/kgK


## Analysis
We will find the convection from the definition of Nusselt number and the empirical formula for
finding the Nusselt number.
$$
\begin{align}
    Nu &= \frac{hL}{k_f} \\
    \overline{Nu}_D &= C_1 C_2 Re_{D, mathrm{max}}^m Pr^{0.36} \left(\frac{Pr}{Pr_s}\right)^{1/4}
\end{align}
$$

We will first check the flow conditions.

In [4]:
# First we need to check where the maximum velocity happens.
diagonal_spacing = np.sqrt(horizontal_spacing**2 + (0.5 * vertical_spacing)**2)
if 2 * (diagonal_spacing - diameter_tube) < (vertical_spacing - diameter_tube):
    velocity_max = vertical_spacing / (2 * (diagonal_spacing - diameter_tube)) * velocity_air
    print(f"Maximum velocity at diagonal path: {velocity_max} m/s")
else:
    velocity_max = vertical_spacing / (vertical_spacing - diameter_tube) * velocity_air
    print(f"Maximum velocity at diagonal path: {velocity_max} m/s")

reynolds_number = fluid_state_air_mean.density * fluid_state_air_mean.characteristic_length * \
                  velocity_max / fluid_state_air_mean.dynamic_viscosity
print(f"Reynolds number is {reynolds_number}.")

Maximum velocity at diagonal path: 12.604026845637584 m/s
Reynolds number is 11989.6620878666.


From the constant table and correction factor table below:

![](images/constants_tube_bank.png)
![](images/correction_factor_tube_bank.png)


In [5]:
# Calculate h
print(f"The ratio between the vertical spacing and horizontal spacing is {vertical_spacing / horizontal_spacing}.")
c1 = 0.35 * (vertical_spacing / horizontal_spacing)**0.2
m = 0.6
c2 = 0.95
prandtdl_number = fluid_state_air_mean.cp * fluid_state_air_mean.dynamic_viscosity / fluid_state_air_mean.k
prandtdl_number_surface = fluid_state_air_mean.cp * fluid_state_air_mean.dynamic_viscosity / fluid_state_air_mean.k
Nu = c1 * c2 * reynolds_number**m * prandtdl_number**0.36 * (prandtdl_number / prandtdl_number_surface)**0.25
h = Nu * fluid_state_air_mean.k / diameter_tube

print(f"Convection coefficient is {h} W/m2K")

The ratio between the vertical spacing and horizontal spacing is 0.9125364431486882.
Convection coefficient is 135.39179091826264 W/m2K


For calculation of heat transfer rate, we need to use the logarithmic mean temperature difference instead of arithmic temperature difference. In addition, we need to estimate the outlet temperature of the stream that externally passes the tubes. We can do that by
$$\Delta T_m = \frac{(T_s - T_i) - (T_s - T_o)}{\ln\left(\frac{T_s - T_i}{T_s - T_o}\right)}$$
$$\frac{T_s - T_o}{T_s - T_i} = exp\left(-\frac{\pi D N \overline{h}}{\rho V N_T S_T c_p}\right)$$

In [6]:
# Calculate q
def get_log_mean_temp_difference(
    temp_surface: float,
    temp_in: float,
    temp_out: float
) -> float:
    return (temp_out - temp_in) / np.log((temp_surface - temp_in) / (temp_surface - temp_out))


def get_temp_out_estimation(
    temp_surface: float,
    temp_in: float,
    diameter_tube: float,
    number_tube: float,
    vertical_spacing: float,
    number_tube_per_row: float,
    velocity: float,
    h: float,
    fluid_state: FluidState
) -> float:
    return temp_surface - (temp_surface - temp_in) * \
        np.exp(-(np.pi * diameter_tube * number_tube * h) / 
               (fluid_state.density * velocity * number_tube_per_row * vertical_spacing * fluid_state.cp))

def get_heat_transfer_tube_bank(
    number_tube: float,
    h: float,
    diameter_tube: float,
    log_mean_temp_diff: float,
    length: float
) -> float:
    return number_tube * h * np.pi * diameter_tube * log_mean_temp_diff * length

number_tube_per_row = 8
number_row = 7
number_tube = number_tube_per_row * number_row

fluid_state_air_in = FluidState(
    fluid=Fluid.AIR,
    temp_k=temp_air_in + 273.15,
    characteristic_length=diameter_tube
)

temp_out = get_temp_out_estimation(
    temp_surface=temp_surface,
    temp_in=temp_air_in,
    diameter_tube=diameter_tube,
    number_tube=number_tube,
    number_tube_per_row=number_tube_per_row,
    vertical_spacing=vertical_spacing,
    velocity=velocity_air,
    h=h,
    fluid_state=fluid_state_air_in
)

log_mean_temp_diff = get_log_mean_temp_difference(
    temp_surface=temp_surface,
    temp_in=temp_air_in,
    temp_out=temp_out
)

q = get_heat_transfer_tube_bank(
    number_tube=number_tube,
    h=h,
    diameter_tube=diameter_tube,
    log_mean_temp_diff=log_mean_temp_diff,
    length=1
)

print(f'Heat transfer rate is {q:.3f} W/m')

Heat transfer rate is 19370.745 W/m
